<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# The UPGMA algorithm

We can now see a python implementation of the UPGMA algorithm, as explained in the video. But like always let us start with this.

In [ ]:
# this is so that we can use print() in python2 like in python3
from __future__ import print_function
# with this, division will behave in python2 like in python3
from __future__ import division

### Needleman and Wunsch's distance

Of course we need the `distance` function that implements Needleman et Wunsch's distance, that we import from a notebook last week:

In [ ]:
from w4_s09_c1_needleman_wunsh_iter import distance

### Input file format

This time we are going to use a slightly different file format, so that we can simply attach a name to each DNA fragment in the set. For this reason, the input file will look like this (these data of course were made up): 

In [ ]:
with open("data/named-species.txt") as input:
    for line in input:
        print(line, end="")

As you can see, each line contains, separated by one a several space characters, a name and a DNA sequence. For this reason we cannot reuse as-is the function that we had written to compute the distances array, and will rewrite this part.

### A few useful tools in python's toolset

##### The `split` method

We will use the `split` on strings, that splits a string into pieces:

In [ ]:
chaine = "BABAAB  ADAD"
chaine.split()

As you can see, without an argument this method does exactly what we need to parse one line in our input file into two parts. 

##### The `remove` method to remove from a list

We will also use the `remove` on lists, which works a bit like `append`, but produces the opposite effect:

In [ ]:
# a list with strings and tuples
l = [ 'a', 'b', (1, 2)]
print("begin:", l)
# we can remove a chain from the list
l.remove('a')
print("middle:", l)
# or remove a tuple
l.remove( (1, 2))
print("end:", l)

##### Check whether a key is present in a dictionary

In order to test whether a key is present in a dictionary or not, we can use the `in` operator:

In [ ]:
d = {'a' : 1, (1, 2) : 'un-deux' }
(1, 2) in d

In [ ]:
'a' in d

In [ ]:
'b' in d

### Data structures

We use the following data structures for implementing our algorithm. 

##### Species

In order to model each species, we will use:
  * either directly a string, for species that are present in the input file, that we call **native species**,
  * or a tuple made out of two species, for species that get created by the algorithm as it owrks, and that we call **synthetic species**.

So for example a species can be 
 * either `spam`,
 * or `('eggs', 'bread')` to model the common ancestor to the `eggs` and `bread` species,
 * or, one further step up, `(('eggs', 'bread'), ('bacon', 'chicken'))`

##### `native_species`

This variable simply is the outcome of parsing our input file; it simply is a dictionary  `name` $\rightarrow$ `dna`.

##### `distances`

Exactly as we had seen in the previous notebook about computing the distances array, this variable will hold the distance between two species, either native or synthetic. Exactly like we had done at that time, we implement this as a dictionary indexed on tuples of species, and we avoid duplications through the introduction of a helper funtion `get_distance`. Of course, this array is initialized only with native species, and further couples get added as we go.

##### `species`

This variable is key, it holds all the species that are still candidates. It gets initialized with all native species, and each a synthetic species gets created:

* the new synthetic species gets added to `species`, and 
* its two components (native or synthetic) get removed.

It is thus `species` that tells when the algorithm is over, when it contains only one species, which at that point is our final result.

### Helpfer functions to access distances

##### Retrieve a distance in the table

Our `distances` data structure takes advantage of the symmetry in the distance function, and so contains only half of the table. As opposed to what we did in the first version of the distances array (sequence 3 this week), we no longer have a total order on species (remember that in this other notebook, species were represented by an integer index).

To state that same thing again but in other words, when looking for the distance between `sp1` and `sp2`, it is not possible to tell *a priori* whether the dictionary will have key `(sp1, sp2)` or key `(sp2, sp1)`. But this is no serious issue, we just need to try both options.

We can thus rewrite the function that looks up the ditances array like this:

In [ ]:
def get_distance(distances, sp1, sp2):
    """
    Searches the distance between sp1 and sp2
    """
    if (sp1, sp2) in distances:
        return distances[ (sp1, sp2) ]
    # otherwise, it must be the other way around
    else:
        return distances[ (sp2, sp1) ]
    # In principle, if everything is correct, 
    # we should not need to consider other cases like sp1 == sp2 

##### Computing minimal distance

We will also need a utility function to compute the minimal distance within all the couples of species still candidate. To this end we will need
 * of course the `distances` variable,
 * and `species` that is the list of species that need to be considered; this is because it is more convenient to keep everything in `distances`, even when a species get solved into a synthetic larger species, and because of that, `species` is strictly smaller than the set of species that appear in `distances`.
 
Here is a possible implementation for the function `minimal_couple`, that returns the couple of species in `species` that have the smallest distance.

In [ ]:
def minimal_couple(distances, species):
    """
    Considers all couples in species x species
    with sp1 != sp2 
    and returns the one with minimal distance
    
    Returns the couple in question
    and not the distance per se because we won't need it
    """
    ### initializations
    # resulting couple
    couple = None
    # smallest value so far
    minimum = None
    # scan all couples
    for sp1 in species:
        for sp2 in species:
            # we consider only the couples that appear as a key 
            # in distances, and this way
            # (*) we avoid the case sp1 == sp2, and
            # (*) each couple is handled only once
            if (sp1, sp2) not in distances:
                continue
            # if minimum is None, we deal with our first couple
            if minimum is None:
                minimum = get_distance(distances, sp1, sp2)
                couple = sp1, sp2
            # otherwise, we select this couple as being the best so far
            # if its distance is smaller than current minimum
            # inférieure au minimum courant
            else:
                candidate = get_distance(distances, sp1, sp2)
                if candidate < minimum:
                    minimum = candidate
                    couple = sp1, sp2
    # let us not forget to return the result
    return couple

### The UPGMA algorithm

We only have to put everything together. For convenience, we allow for an additional input parameter `verbose`, and when this is `True` we produce a more verbose output:

In [ ]:
def UPGMA(filename, verbose=False):
    """
    Reads a file that contains on each line:
    the species name and its DNA 
    
    The optional paramater verbose allows to make the ouput
    more verbose and to illustrate how the algorithm is progressing

    Computes a distances array, then implements UPGMA
    
    Returns the filiation tree, as a tuple of species 
    (each being itself a species name or a tuple)
    """
    
    native_species = {}

    # Read the file
    with open(filename) as input:
        for line in input:
            # split line into name and sequence
            name, dna = line.split()
            # on mémorise dans native_species
            native_species[name] = dna
    
    # compute the distances array
    distances = {}

    # primarily like in the previous sequence
    for sp1, dna1 in native_species.items():
        for sp2, dna2 in native_species.items():
            # ignore diagonal couples
            if sp1 == sp2:
                continue
            # the only trick here is to ignore that couple if its symmetic 
            # is already in distances
            if (sp2, sp1) in distances:
                continue
            distances[sp1, sp2] = distance(dna1, dna2)
    
    # initialize speices as the initial list of keys 
    species = native_species.keys()
    
    # verbosity
    if verbose:
        print(10*'+', 'Initial distances')
        print(distances )

    # this is where interesting things happen
    # it is expected that species will progressively shrink until it only
    # contains one synthetic species, that holds the filiation tree
    while len(species) > 1:
        # the closest couple of species
        closer1, closer2 = minimal_couple(distances, species)
        # we can remove them from our radar
        species.remove(closer1)
        species.remove(closer2)
        # and create a synthetic species out of them
        new_species = closer1, closer2
        # we need to recompute the distances between this species
        # and the ones still candidate
        # as in the slide:
        # dist(F,C),A = (dist F,A + dist C,A) / 2 
        for sp in species:
            distances[ sp, new_species ] = \
              (get_distance(distances, closer1, sp) + \
               get_distance(distances, closer2, sp)) / 2
        # we can now add the new species in the set of candidates
        # for the next round
        species.append(new_species)
        # verbosity
        if verbose:
            print(10*'=', "species = ", species)
            print(distances)
    # the result is the single element in species
    return species[0]

### On a simple example

With the data from `data/named-species.txt`, again:

In [ ]:
with open("data/named-species.txt") as input:
    for line in input:
        print(line, end="")

We obtain this result:

In [ ]:
UPGMA("data/named-species.txt")

Or, with a verbose output:

In [ ]:
UPGMA("data/named-species.txt", True)